<img src="https://www.who.int//images/default-source/infographics/who-emblem.png" width="100" align="left">

# Querying WHO Data
    


## This data is obtained from the GHO (Global Health Observatory)

### Darío Ramírez Naicipa

### Debido a la gran cantidad de información de salud contenida en el Observatorio de la WHO vamos a seguir algunos pasos para segmentar las consultas hacia el API

In [81]:
#Librerias 
import pandas as pd
import requests
import ipywidgets as widgets
from IPython.display import display
import matplotlib.pyplot as plt

### 1. Consultar las *Dimensiones* disponibles

In [82]:
#Datos del conexion al API
url_dimensiones = "http://apps.who.int/gho/athena/api/"
parametros = {'format': 'json'}

#LLamado del API
response_dim = requests.get(url_dimensiones,params=parametros)
datos_dimensiones = response_dim.json()

#Construccion DF Pandas
df_dimensiones=pd.DataFrame(datos_dimensiones["dimension"])

#Render del combo de seleccion
ddl_dimension = widgets.Dropdown(options=df_dimensiones["display"])
display(ddl_dimension)

Dropdown(options=('Indicator', 'Indicator Categories', 'Data Source', 'Child cause', 'ICD10CAUSEGROUP', 'ICD10…

### 2. Consultar los codigos disponibles para las  *Dimensiónes*

In [83]:
#Funcion para extraer las categorias de las Dimensiones
def extract_cat(lista_cat):
    df_tmp = pd.DataFrame(lista_cat)
    #df_tmp.query('category=="CATEGORY"')["value"].iloc[0]
    if "category" in df_tmp.columns:
        value = df_tmp.query('category=="CATEGORY"')["value"].values
        return value
    else : return None

In [84]:
#Se recuperan el codigo de la dimension
dimension_code = list(df_dimensiones[df_dimensiones["display"] == ddl_dimension.value]["label"])[0]

#Datos del conexion al API
url_datos_dimension = f"http://apps.who.int/gho/athena/api/{dimension_code}"
parametros = {'format': 'json'}

#LLamado del API
response_datos_dimension = requests.get(url_datos_dimension,params=parametros)
datos_dimension = response_datos_dimension.json()
datos_dimension

#DF Atributos
df_atribute = pd.DataFrame(datos_dimension["attribute"])

#DF Categorias
df_codes =pd.DataFrame(datos_dimension["dimension"][0]["code"]).sort_values(by=["display"])
df_codes["category"]=df_codes["attr"].map(extract_cat)

#Render del combo de seleccion
ddl_datos_dimension = widgets.Dropdown(options=df_codes['display'])
display(ddl_datos_dimension)

Dropdown(options=('ALL CAUSES', 'Acute lower respiratory infections', 'Birth asphyxia and birth trauma', 'Cong…

### 3. Filtrado de Datos

In [85]:
#Se recuperan el codigo de la dimension
filter_code = df_codes.query(f"display=='{ddl_datos_dimension.value}'")["label"].iloc[0]

#Datos del conexion al API
url_datos_final = f"http://apps.who.int/gho/athena/api/{dimension_code}/{filter_code}/YEAR"
parametros = {'format': 'json'}
url_datos_final

#LLamado del API
response_datos_final = requests.get(url_datos_final,params=parametros)
datos_final = response_datos_final.json()
#datos_final

#DF
df_final_attribute = pd.DataFrame(datos_final["attribute"])
df_final_dimension = pd.DataFrame(datos_final["dimension"])
df_final_fact = pd.DataFrame(datos_final["fact"])


### 4. Procesamiento de Dimensiones y Hechos descargados

In [86]:
#crea estructura hechos
df_final_fact_dim = pd.DataFrame(df_final_fact["Dim"][0]).set_index("category",drop=True)
df_final_fact_dim = df_final_fact_dim.T
df_final_fact_dim["fact_id"]=None
df_final_fact_dim = df_final_fact_dim.drop("code",axis=0)

for item in df_final_fact.iterrows():
    t = pd.DataFrame(item[1]["Dim"]).set_index("category",drop=True)
    t = t.T #trasponer
    t["fact_id"] = item[1]["fact_id"]
    df_final_fact_dim = df_final_fact_dim.append(t)
    
#df_final_fact_dim

In [87]:
#crea estructura medidas
df_final_fact_measures = pd.DataFrame(df_final_fact,columns=["fact_id","value"])

for item in df_final_fact_measures.iterrows():
    t = pd.DataFrame(item[1]["value"].items()).set_index(0,drop=True)
    t = t.T #trasponer
    t["fact_id"] = item[1]["fact_id"]
    df_final_fact_measures = df_final_fact_measures.append(t)

df_final_fact_measures = df_final_fact_measures[df_final_fact_measures["value"].isnull() == True]
df_final_fact_measures = df_final_fact_measures.drop(columns=["value"],axis=0)

#df_final_fact_measures


In [111]:
#url_datos_final

#df_final_attribute
df_final_dimension
#df_final_fact

#df_final_fact_measures
#df_final_fact_dim

#df_analisys

,label,display,isMeasure,code
0,GHO,Indicator,True,"[{'label': 'MORT_100', 'display': 'Number of d..."
1,GBDCHILDCAUSES,Child cause,False,"[{'label': 'CH9', 'display': 'Acute lower resp..."
2,PUBLISHSTATE,PUBLISH STATES,False,"[{'label': 'PUBLISHED', 'display': 'Published'..."
3,YEAR,Year,False,"[{'label': '2000', 'display': '2000', 'display..."
4,REGION,WHO region,False,"[{'label': 'AFR', 'display': 'Africa', 'displa..."
5,COUNTRY,Country,False,"[{'label': 'AFG', 'display': 'Afghanistan', 'd..."
6,AGEGROUP,Age Group,False,"[{'label': 'DAYS0-27', 'display': '0-27 days',..."
7,SEX,Sex,False,"[{'label': 'BTSX', 'display': 'Both sexes', 'd..."
8,MGHEREG,Region,False,"[{'label': 'MDG_CCA', 'display': 'Caucasus and..."


In [90]:
#Se quitan las columnas que se tranformaron.
#df_final_fact = df_final_fact.drop(columns="value",axis=1)
#df_final_fact = df_final_fact.drop(columns="Dim",axis=1)
#df_final_fact = df_final_fact.drop(columns="published",axis=1)

#merge de multiples DataFrames
df_analisys = pd.merge(df_final_fact,df_final_fact_dim, how="inner", on="fact_id").merge(df_final_fact_measures,how="inner",on="fact_id")

df_analisys

,fact_id,dataset,effective_date,end_date,AGEGROUP,REGION,COUNTRY,SEX,GHO,YEAR,GBDCHILDCAUSES,PUBLISHSTATE,MGHEREG,display,numeric,low,high,stderr,stddev
0,9028041,GCD,2014-10-01,2900-12-31,MONTHS1-59,EUR,ALB,BTSX,MORT_100,2013,CH9,PUBLISHED,NaN,52,51.84025,None,None,None,None
1,9028177,GCD,2014-10-01,2900-12-31,YEARS0-4,AMR,ARG,BTSX,MORT_100,2013,CH9,PUBLISHED,NaN,844,844.0131,None,None,None,None
2,9028626,GCD,2014-10-01,2900-12-31,MONTHS1-59,SEAR,BGD,BTSX,MORT_100,2013,CH9,PUBLISHED,NaN,14729,14728.6,None,None,None,None
3,9029392,GCD,2014-10-01,2900-12-31,YEARS0-4,WPR,CHN,BTSX,MORT_100,2013,CH9,PUBLISHED,NaN,31867,31867.08,None,None,None,None
4,9030830,GCD,2014-10-01,2900-12-31,DAYS0-27,AFR,GHA,BTSX,MORT_100,2013,CH9,PUBLISHED,NaN,1280,1279.808,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5595,9481900,GCD,2014-10-06,2900-12-31,MONTHS1-59,NaN,NaN,BTSX,MORT_300,2012,CH9,PUBLISHED,UNICEF_WCARO,19.9,19.9257,None,None,None,None
5596,9481901,GCD,2014-10-06,2900-12-31,YEARS0-4,NaN,NaN,BTSX,MORT_300,2012,CH9,PUBLISHED,UNICEF_WCARO,15.6,15.60191,None,None,None,None
5597,9481944,GCD,2014-10-06,2900-12-31,DAYS0-27,NaN,NaN,BTSX,MORT_300,2013,CH9,PUBLISHED,UNICEF_WCARO,6.4,6.42983,None,None,None,None
5598,9481945,GCD,2014-10-06,2900-12-31,MONTHS1-59,NaN,NaN,BTSX,MORT_300,2013,CH9,PUBLISHED,UNICEF_WCARO,19.5,19.50877,None,None,None,None


### 5. Tablas de Agrupamiento

In [109]:
#ajustar tipo de dato para las operaciones de agrupacion
df_analisys["display"] = pd.to_numeric(df_analisys["display"])

column_list = ["REGION","SEX"]
df_g = df_analisys.groupby(column_list).sum()
df_g

,,display
REGION,SEX,
AFR,BTSX,2283265.0
AMR,BTSX,159897.0
EMR,BTSX,519730.0
EUR,BTSX,101829.0
SEAR,BTSX,1561285.0
WPR,BTSX,555403.0


### 6. Probando Graficos

In [112]:
#df_g.plot.barh(stacked=True,figsize=(20,20),title=f"{ddl_dimension.value} - {ddl_datos_dimension.value}")
#df_g.plot(figsize=(20,10),title=f"{ddl_dimension.value} - {ddl_datos_dimension.value}")
#df_g.plot.bar(figsize=(20,10)title=f"{ddl_dimension.value} - {ddl_datos_dimension.value}")
#df_g.plot.pie(subplots=True,figsize=(10,10),title=f"{ddl_dimension.value} - {ddl_datos_dimension.value}")